In [1]:
import os
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
plt.style.use(os.path.join(mpl.get_configdir(), 'latexstyle.mplstyle'))

In [50]:
from scipy import stats
# x2= df.from_dict(labelsDict)[differentLabelTypes[2]]
# x1= df.from_dict(labelsDict)[differentLabelTypes[1]]
def quantile_plot(x, **kwargs):
    quantiles, xr = stats.probplot(x, fit=False)
    plt.scatter(xr, quantiles, **kwargs)
    
def qqplot(x, y, **kwargs):
    _, xr = stats.probplot(x, fit=False)
    _, yr = stats.probplot(y, fit=False)
    plt.scatter(xr, yr, **kwargs)

# quantile_plot(x1)
# quantile_plot(x2)
# _=plt.title('Quantile Plots for Labels')
#todo add legends and a 45% degree line

In [51]:
passport = '/media/ak/My Passport/Data/FinDataReal/LabelsAlternateOne/'


In [6]:
ticker = 'BARC.L'
filename = "".join((passport,ticker.split(".")[0],'_AlternateLabels.pickle'))
with open(filename, 'rb') as handle:
    data =pickle.load(handle)


###  label types ###

In [52]:
df = data[list(data.keys())[3]]
differentLabelTypes=df.filter(like='label', axis=1).columns.values
differentLabelTypes.shape

(5,)

In [12]:
import collections
from collections import OrderedDict
dataLabels = OrderedDict(dict)
for key in list(data.keys()):
    print(key)
    df = data[key]
    differentLabelTypes=df.filter(like='label', axis=1).columns.values
    for Label in list(differentLabelTypes.shape) ataLabels[key][differentLabelTypes[]]
    
#     print(str(differentLabelTypes[0]))
#     print(df[differentLabelTypes[0]].value_counts())
#     print('----------------------***-----------------')
#     print(str(differentLabelTypes[1]))
#     print(df[differentLabelTypes[1]].value_counts())    
#     print('----------------------')

20170116
label_MovAv__window_2__thres_arbitrary__0.0
 0.0    2394
 1.0    1404
-1.0    1355
Name: label_MovAv__window_2__thres_arbitrary__0.0, dtype: int64
----------------------***-----------------
label_MovAv__window_5__thres_arbitrary__0.1
 0.0    4670
-1.0     271
 1.0     209
Name: label_MovAv__window_5__thres_arbitrary__0.1, dtype: int64
----------------------
20170117
label_MovAv__window_2__thres_arbitrary__0.0
 0.0    3944
-1.0    2230
 1.0    2207
Name: label_MovAv__window_2__thres_arbitrary__0.0, dtype: int64
----------------------***-----------------
label_MovAv__window_5__thres_arbitrary__0.1
 0.0    7693
 1.0     395
-1.0     290
Name: label_MovAv__window_5__thres_arbitrary__0.1, dtype: int64
----------------------
20170118
label_MovAv__window_2__thres_arbitrary__0.0
 0.0    4540
-1.0    1892
 1.0    1838
Name: label_MovAv__window_2__thres_arbitrary__0.0, dtype: int64
----------------------***-----------------
label_MovAv__window_5__thres_arbitrary__0.1
 0.0    7799
-1.0  

label_MovAv__window_2__thres_arbitrary__0.0
 0.0    1748
-1.0     517
 1.0     498
Name: label_MovAv__window_2__thres_arbitrary__0.0, dtype: int64
----------------------***-----------------
label_MovAv__window_5__thres_arbitrary__0.1
 0.0    2663
-1.0      58
 1.0      39
Name: label_MovAv__window_5__thres_arbitrary__0.1, dtype: int64
----------------------
20170831
label_MovAv__window_2__thres_arbitrary__0.0
 0.0    2142
 1.0     595
-1.0     585
Name: label_MovAv__window_2__thres_arbitrary__0.0, dtype: int64
----------------------***-----------------
label_MovAv__window_5__thres_arbitrary__0.1
 0.0    3214
-1.0      53
 1.0      52
Name: label_MovAv__window_5__thres_arbitrary__0.1, dtype: int64
----------------------
20170901
label_MovAv__window_2__thres_arbitrary__0.0
 0.0    1896
 1.0     522
-1.0     481
Name: label_MovAv__window_2__thres_arbitrary__0.0, dtype: int64
----------------------***-----------------
label_MovAv__window_5__thres_arbitrary__0.1
 0.0    2749
-1.0      75
 1

In [45]:
# pd.concat([data[k].set_index(k) for k in list(data.keys())], axis =0)
df = pd.concat([pd.DataFrame({label: data[k][label].tolist() for label in differentLabelTypes}, index = [k]*len(data[k])) for k in list(data.keys())], axis =0)

In [47]:
from scipy import stats
x2= df.from_dict(labelsDict)[differentLabelTypes[2]]
x1= df.from_dict(labelsDict)[differentLabelTypes[1]]

,label_MovAv__window_2__thres_arbitrary__0.0,label_MovAv__window_5__thres_arbitrary__0.1,label_PrMov__window_2__thres_arbitrary__0.5,label_EndPo__window_9__thres_arbitrary__0.1,label_MovAv__window_20__thres_arbitrary__0.8
20170116,-1.0,-1.0,0.0,-1.0,0.0
20170116,-1.0,-1.0,0.0,-1.0,0.0
20170116,-1.0,-1.0,0.0,-1.0,0.0
20170116,-1.0,-1.0,0.0,-1.0,0.0
20170116,0.0,-1.0,0.0,-1.0,0.0
20170116,-1.0,-1.0,0.0,-1.0,0.0
20170116,-1.0,-1.0,0.0,1.0,0.0
20170116,-1.0,-1.0,0.0,-1.0,0.0
20170116,-1.0,0.0,0.0,0.0,0.0
20170116,1.0,0.0,0.0,0.0,0.0
